# 필요한 라이브러리 import

In [1]:
import csv
import pandas as pd
import os
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
import warnings
import pickle
plt.rc('font',family='Malgun Gothic')
plt.rc('font',size=10)
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import load_model
import tensorflow as tf
import joblib
warnings.filterwarnings(action='ignore')

In [2]:
with open('./models/defender/defender_qt.pickle','rb') as f:
    defender_po = pickle.load(f)

defender_model = load_model('./models/defender/defender_nn.h5')

premier_df = pd.read_csv('./datas/NEW_FM_DATA_MERGED/2021-2022/2021-2022-PremierLeague.csv')

In [3]:
premier_df.rename(columns={'Cr C/A':'크로스 성공률','FA':'파울 획득','Int/90':'인터셉트/90','공중 A/90':'공중볼시도/90','K Ps/90':'키패스/90','Cr A/90':'크로스 시도/90'},inplace=True)
premier_df = premier_df.astype({'파울 획득':np.int64,'태클 성공률':np.int64})

In [4]:
defender_df = premier_df[premier_df['포지션']=='수비수']

In [5]:
# PSTL = Pass sucess try linear

defender_df['PSTL'] = (defender_df['경기당 패스 성공/90']) + (defender_df['경기당 패스 시도/90']*1.2)

In [6]:
independent_value = {'PSTL', '태클 성공/90', '헤더 성공/90'}

dependent_value = {'평균획득승점/경기'}

In [7]:
defender_df.loc[:,independent_value] = defender_po.transform(defender_df.loc[:,independent_value])

In [8]:
defender_df = defender_df.reset_index(drop=True)

In [9]:
defender_df['예측PPG']=defender_model.predict(defender_df.loc[:,independent_value])

In [10]:
defender_df.sort_values(by='예측PPG',ascending=False).head(5)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기,PSTL,예측PPG
17,Rodri,2021-2022,수비수,맨시티,잉글랜드 프리미어 리그,3,3,0,24,10,...,64.34,60.11,0.79,2.690265,0.23,-10.376616,0.11,1.82,56.400000,2.267734
82,William Carvalho,2021-2022,수비수,토트넘,잉글랜드 프리미어 리그,1,5,0,10,4,...,63.98,60.78,0.84,0.955752,0.12,-10.632462,0.06,1.79,56.504846,2.259411
77,Luka Milivojević,2021-2022,수비수,팰리스,잉글랜드 프리미어 리그,1,2,0,14,10,...,63.39,58.55,0.74,1.168142,0.05,-10.600367,0.14,1.30,55.210573,2.229390
2,Diego Carlos,2021-2022,수비수,뉴캐슬,잉글랜드 프리미어 리그,1,1,0,9,3,...,62.12,59.18,0.06,0.053097,0.18,-10.058414,0.00,1.76,54.816740,2.229138
8,Jonny Evans,2021-2022,수비수,레스터,잉글랜드 프리미어 리그,0,1,0,6,3,...,62.08,58.99,0.31,-0.761062,0.00,-9.988721,0.00,1.05,54.711894,2.225870


In [11]:
defender_min = np.min(defender_df['예측PPG'].tolist())
defender_min

1.6969858407974243

In [12]:
defender_df['WAR'] = defender_df['예측PPG'].map(lambda x : x-defender_min+1)

In [13]:
defender_df.loc[:,independent_value] = defender_po.inverse_transform(defender_df.loc[:,independent_value])

In [14]:
defender_war = defender_df.sort_values(by='WAR',ascending=False).copy()
defender_war = defender_war.reset_index(drop=True)

In [16]:
defender_war[['이름','PSTL','태클 성공/90', '헤더 성공/90','WAR']].sort_values(by='WAR',ascending=False).head(10)

,이름,PSTL,태클 성공/90,헤더 성공/90,WAR
0,Rodri,137.318,2.43,5.82,1.570748
1,William Carvalho,137.556,1.45,3.03,1.562425
2,Luka Milivojević,134.618,1.57,3.38,1.532404
3,Diego Carlos,133.724,0.94,9.29,1.532153
4,Jonny Evans,133.486,0.48,10.05,1.528884
5,Pontus Jansson,130.324,0.97,9.97,1.497442
6,Willy Boly,130.276,0.58,9.57,1.494102
7,Axel Tuanzebe,130.224,0.70,9.31,1.493606
8,Joachim Andersen,130.196,0.63,9.02,1.492339
9,Kurt Zouma,129.070,0.87,9.68,1.482929
